In [22]:
# 새로운 이미지로 테스트 진행
!pip install torch opencv-python

In [24]:
%cd C:\Users\gram\geekhub\hongchae
!git clone https://github.com/ultralytics/yolov5

C:\Users\gram\geekhub\hongchae


fatal: destination path 'yolov5' already exists and is not an empty directory.


In [25]:
%cd yolov5
!pip install -r requirements.txt

C:\Users\gram\geekhub\hongchae\yolov5


In [26]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 데이터 경로 설정
data_path = "C:\\Users\\gram\\geekhub\\hongchae\\CASIA-Iris-Interval"
output_path = "C:\\Users\\gram\\geekhub\\hongchae\\yolov5"

# 학습용 및 검증용 폴더 생성
train_img_dir = os.path.join(output_path, 'images/train')
val_img_dir = os.path.join(output_path, 'images/val')
train_label_dir = os.path.join(output_path, 'labels/train')
val_label_dir = os.path.join(output_path, 'labels/val')

os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# 이미지와 라벨 파일 목록 가져오기
img_files = []
label_files = []

for root, _, files in os.walk(data_path):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            img_files.append(os.path.join(root, file))
            label_files.append(os.path.join(root, file + '.txt'))

# 데이터셋을 학습용과 검증용으로 분리
train_imgs, val_imgs, train_labels, val_labels = train_test_split(img_files, label_files, test_size=0.2, random_state=42)

# 파일 이동
for img, label in zip(train_imgs, train_labels):
    shutil.copy(img, train_img_dir)
    shutil.copy(label, train_label_dir)

for img, label in zip(val_imgs, val_labels):
    shutil.copy(img, val_img_dir)
    shutil.copy(label, val_label_dir)

print("데이터셋 분리 완료")

데이터셋 분리 완료


In [27]:
import os

# 경로 설정
train_label_dir = "C:\\Users\\gram\\geekhub\\hongchae\\yolov5\\labels\\train"
val_label_dir = "C:\\Users\\gram\\geekhub\\hongchae\\yolov5\\labels\\val"

def rename_files_in_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".jpg.txt"):
            # 새로운 파일 이름 생성 (".jpg" 부분 제거)
            new_filename = filename.replace(".jpg.txt", ".txt")
            old_filepath = os.path.join(directory, filename)
            new_filepath = os.path.join(directory, new_filename)
            
            # 파일 이름 변경
            os.rename(old_filepath, new_filepath)
            print(f"Renamed: {old_filepath} -> {new_filepath}")

# train과 val 디렉토리에서 파일 이름 변경
rename_files_in_directory(train_label_dir)
rename_files_in_directory(val_label_dir)

print("파일 이름 변경 완료")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:\\Users\\gram\\geekhub\\hongchae\\yolov5\\labels\\train\\S1001L01.jpg.txt' -> 'C:\\Users\\gram\\geekhub\\hongchae\\yolov5\\labels\\train\\S1001L01.txt'

In [28]:
import os

# data.yaml 파일 내용
data_yaml_content = """
train: C:\\Users\\gram\\geekhub\\hongchae\\yolov5\\images\\train
val: C:\\Users\\gram\\geekhub\\hongchae\\yolov5\\images\\val

nc: 1  # 클래스 개수 (홍채 클래스)
names: ['iris']
"""

# data.yaml 파일 생성 경로
data_yaml_path = "C:\\Users\\gram\\geekhub\\hongchae\\yolov5\\data.yaml"

# 디렉토리가 존재하지 않는 경우 생성
os.makedirs(os.path.dirname(data_yaml_path), exist_ok=True)

# data.yaml 파일 생성
with open(data_yaml_path, 'w') as file:
    file.write(data_yaml_content)

print(f"data.yaml 파일이 생성되었습니다: {data_yaml_path}")

data.yaml 파일이 생성되었습니다: C:\Users\gram\geekhub\hongchae\yolov5\data.yaml


In [79]:
import torch
import cv2
import numpy as np
import sys
from pathlib import Path

# YOLOv5 저장소 경로 추가
sys.path.append('C:\\Users\\gram\\geekhub\\hongchae\\yolov5')

from models.common import DetectMultiBackend
from utils.general import non_max_suppression
from utils.torch_utils import select_device

# scale_coords 함수 정의
def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
    if ratio_pad is None:
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2
    else:
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2]] -= pad[0]
    coords[:, [1, 3]] -= pad[1]
    coords[:, :4] /= gain
    coords[:, 0].clamp_(0, img0_shape[1])
    coords[:, 1].clamp_(0, img0_shape[0])
    coords[:, 2].clamp_(0, img0_shape[1])
    coords[:, 3].clamp_(0, img0_shape[0])
    return coords

# YOLOv5 모델 로드
model_path = "C:\\Users\\gram\\geekhub\\hongchae\\yolov5\\runs\\train\\exp\\weights\\best.pt"
device = select_device('cpu')
model = DetectMultiBackend(weights=str(model_path), device=device)

# 이미지 로드
image_path = "C:\\Users\\gram\\geekhub\\hongchae\\161.jpg"  # 2번 이미지 경로
image = cv2.imread(image_path)

if image is None:
    print(f"Error: Unable to load image at {image_path}")
else:
    print(f"Loaded image with shape: {image.shape}")

# 이미지 전처리
img_size = 640
image_resized = cv2.resize(image, (img_size, img_size))
print(f"Resized image to: {image_resized.shape}")

# BGR to RGB and HWC to CHW
img = image_resized[:, :, ::-1].transpose(2, 0, 1)
print(f"Image shape after channel conversion and transpose: {img.shape}")

img = np.ascontiguousarray(img)
img = torch.from_numpy(img).to(device)
img = img.float()
img /= 255.0
print(f"Image shape after conversion to tensor and normalization: {img.shape}")

if img.ndimension() == 3:
    img = img.unsqueeze(0)
    print(f"Image shape after adding batch dimension: {img.shape}")

# 추론
pred = model(img, augment=False, visualize=False)
print(f"Prediction shape: {pred[0].shape} - First few predictions: {pred[0][:5]}")
pred = non_max_suppression(pred, 0.25, 0.45, None, False, max_det=1000)
print(f"NMS done. Number of detections: {len(pred)}")

# 바운딩 박스 그리기 및 객체 인식 확인
object_detected = False
for i, det in enumerate(pred):
    if len(det):
        object_detected = True
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], image.shape).round()

        for *xyxy, conf, cls in reversed(det):
            if conf > 0.3:  # 신뢰도 임계값을 낮춤
                print(f"Drawing box with confidence {conf:.2f} at [{int(xyxy[0])}, {int(xyxy[1])}, {int(xyxy[2])}, {int(xyxy[3])}]")
                cv2.rectangle(image, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
                label = f"Iris: {conf:.2f}"
                cv2.putText(image, label, (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

if not object_detected:
    print("No objects detected. Potential reasons:")
    print("- The confidence threshold (0.3) may be too high.")  
    print("- The model may not be well trained for the given image.")
    print("- The image may not have clear features for detection.")
    print("- The object size or scale might be out of the expected range for the model.")

# 결과 이미지 표시 및 저장
cv2.imshow("Iris Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

output_image_path = "C:\\Users\\gram\\geekhub\\hongchae\\161result.jpg"
cv2.imwrite(output_image_path, image)
print(f"결과 이미지가 저장되었습니다: {output_image_path}")


YOLOv5  v7.0-318-gc0380fd8 Python-3.9.6 torch-2.3.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs


Loaded image with shape: (314, 314, 3)
Resized image to: (640, 640, 3)
Image shape after channel conversion and transpose: (3, 640, 640)
Image shape after conversion to tensor and normalization: torch.Size([3, 640, 640])
Image shape after adding batch dimension: torch.Size([1, 3, 640, 640])
Prediction shape: torch.Size([1, 25200, 6]) - First few predictions: tensor([[[4.08089e+00, 4.50542e+00, 3.74383e+01, 4.00486e+01, 1.88767e-07, 9.99981e-01],
         [1.18798e+01, 3.87581e+00, 3.80020e+01, 3.89157e+01, 4.13509e-07, 9.99981e-01],
         [2.08618e+01, 3.80554e+00, 3.81466e+01, 3.84305e+01, 5.88892e-07, 9.99979e-01],
         ...,
         [5.61741e+02, 6.02858e+02, 1.70941e+02, 9.30582e+01, 1.00356e-06, 9.99980e-01],
         [5.79026e+02, 6.03502e+02, 1.36751e+02, 9.07446e+01, 9.60743e-07, 9.99978e-01],
         [6.06178e+02, 6.07295e+02, 1.13585e+02, 9.99413e+01, 1.01917e-07, 9.99978e-01]]])
NMS done. Number of detections: 1
Drawing box with confidence 0.37 at [106, 104, 245, 225